In [6]:
import os
import pandas as pd
from itables import init_notebook_mode
import yaml
import time
from datetime import datetime


from IPython.display import display, clear_output


import tensorboard as tb
import numpy as np
import os
import tensorboard.backend.event_processing.event_accumulator as ea


def date_format(time_s):
    days = f"{int(time_s // (24 * 60**2))} days  "
    hours, remainder = divmod(time_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    formatted_time = '{}:{:02d}:{:02d}'.format(int(hours), int(minutes), int(seconds))
    return formatted_time

def read_tensorboard_elasped_time(tensorboard_dir):
    event_files = os.listdir(tensorboard_dir + '/train')
    # sort by created date
    event_files.sort(key=lambda x: os.path.getctime(os.path.join(tensorboard_dir + '/train', x)))

    event_file = event_files[-1]
    # print(event_file)
    event_acc = ea.EventAccumulator(tensorboard_dir + '/train/' + event_file)   
    event_acc.Reload()
    # Show all tags in the log file
    a = event_acc.Tensors('epoch_loss')

    wall_times = [x.wall_time for x in a]
    elapsed = np.max(wall_times) - np.min(wall_times)
    return elapsed
    # # convert elapsed time to hours, minutes, seconds
    # days = f"{int(elapsed // (24 * 60**2))} days  "
    # hours, remainder = divmod(elapsed, 3600)
    # minutes, seconds = divmod(remainder, 60)
    # formatted_time = '{}:{:02d}:{:02d}'.format(int(hours), int(minutes), int(seconds))
    # return formatted_time


STATUS_DIR = "/home/hpinkard_waller/GitRepos/microscoBayes/experiments/config_files/"
SAVING_DIR_ROOT = "/home/hpinkard_waller/models/"


names = []

complete = os.listdir(STATUS_DIR + 'complete')
training = os.listdir(STATUS_DIR + 'training')
pending = os.listdir(STATUS_DIR + 'pending')
staging = os.listdir(STATUS_DIR + 'staging')
abandoned = os.listdir(STATUS_DIR + 'abandoned')


config_files = complete + training + pending + staging + abandoned
statuses = len(complete) * ["complete"] + len(training) * ["training"] + len(pending) * ["pending"] + len(staging) * ["staging"] + len(abandoned) * ["abandoned"] 


# Read stuff from its config file
tensorboard_dirs = []
config_paths = []
dates = []
elapsed_times_tensorboard = []
elapsed_times_config = []
attempts = []

# hyperparameters
all_marker_training = []
arch = []
batch_size = []
density_output = []
learning_rate = []
max_epochs = []
num_mixture_components = []
overshoot_epochs = []
single_marker_early_stopping = []
single_marker_training = []


for config_file, status in zip(config_files, statuses):
    config_file_path = STATUS_DIR + status + '/' + config_file
    with open(config_file_path, "r") as stream:
        config = yaml.safe_load(stream)
        m_time = os.path.getmtime(config_file_path)
        
    if 'training' not in config or 'start_date' not in config['training']:
        dates.append(datetime.fromtimestamp(m_time).strftime("%Y-%m-%d"))
    else:
        dates.append(config['training']['start_date'])
    if 'training' not in config or 'attempt_number' not in config['training']:
        attempts.append('NA')
    else:
        attempts.append(int(config['training']['attempt_number']))

    if 'training' in config:
        if 'tensorboard_dir' not in config['training']:
            tensorboard_dirs.append('NA')
        else:
            tensorboard_dirs.append(config['training']['tensorboard_dir'])
    else:
        tensorboard_dirs.append('NA')
    
    elapsed_times_tensorboard.append(None if 'training' not in config else read_tensorboard_elasped_time(config['training']['tensorboard_dir']))
    elapsed_times_config.append(None if 'training' not in config else config['training']['elapsed'])

    
    # hyperparameters
    all_marker_training.append(config['hyperparameters']['all_marker_training'])
    arch.append(config['hyperparameters']['arch'])
    batch_size.append(config['hyperparameters']['batch_size'])
    density_output.append(config['hyperparameters']['density_output'])
    learning_rate.append(config['hyperparameters']['learning_rate'])
    max_epochs.append(config['hyperparameters']['max_epochs'])
    num_mixture_components.append(config['hyperparameters']['num_mixture_components'])
    overshoot_epochs.append(config['hyperparameters']['overshoot_epochs'])
    single_marker_early_stopping.append(config['hyperparameters']['single_marker_early_stopping'])
    single_marker_training.append(config['hyperparameters']['single_marker_training'])


    if elapsed_times_tensorboard[-1] is not None:
        elapsed_times_tensorboard[-1] = date_format(elapsed_times_tensorboard[-1])
    if elapsed_times_config[-1] is not None and elapsed_times_config[-1] != 'NA':
        elapsed_times_config[-1] = date_format(elapsed_times_config[-1])


    config_paths.append(STATUS_DIR.replace(os.path.expanduser('~'), '') + status + '/' + config_file)


d = {
    "date": dates,  
    "name": [cf.replace('_', " ") for cf in config_files],

    # "all_marker_training": all_marker_training,
    # "arch": arch,
    # "batch size": batch_size,
    # "density output": density_output,
    # "learning rate": learning_rate,
    # "max epochs": max_epochs,
    # "num mixture components": num_mixture_components,
    # "overshoot_epochs": overshoot_epochs,
    # "single marker early stopping": single_marker_early_stopping,
    # "single marker training": single_marker_training,

     "status": statuses, 
     "attempts": attempts, 
      "elapsed_time (tensorboard)": elapsed_times_tensorboard,
      "elapsed_time (config)": elapsed_times_config,
    #  "config": config_paths, 
    #  "tensorboard": tensorboard_dirs}
}
df = pd.DataFrame(data=d)
# df.style.set_properties(subset=['config_files'], **{'min-width': '500px'})
pd.options.display.min_rows = 100



df['status'] = pd.Categorical(df['status'], ["complete", "training", "pending", "staging", "abandoned"])
df['date'] = pd.to_datetime(df['date'])
# dont show time, only show the date
df['date'] = df['date'].dt.date
df = df.sort_values(by=['status', 'date'])

def stylize(x):
    if x == 'complete':
        return 'color: green'
    elif x == 'training':
        return 'color: orange'
    elif x == 'pending':
        return 'color: blue'
    elif x == 'staging':
        return 'color: lightblue'
    elif x == 'abandoned':
        return 'color: gray'
    elif x == 'NA':
        return 'color: gray'
    return None
    

df = df.style.applymap(stylize)  

    
display(df)


,date,name,status,attempts,elapsed_time (tensorboard),elapsed_time (config)
0,2023-03-02,output density type gaussian.yaml,complete,1,4:53:34,NA
1,2023-03-02,Different input contrasts DPC solved trial1.yaml,complete,1,4:53:46,5:09:44
2,2023-03-02,Different input contrasts DPC raw and solved trial1.yaml,complete,1,7:02:03,7:26:03
3,2023-03-02,marginals 1d scalar density hparams all markers.yaml,complete,1,3:10:41,NA
5,2023-03-02,output density type logistic.yaml,complete,1,4:36:41,NA
7,2023-03-02,marginals 1d scalar density hparams CD3 early stopping.yaml,complete,1,3:15:36,NA
8,2023-03-02,Different input contrasts Brightfield trial1.yaml,complete,1,9:21:06,9:45:20
9,2023-03-02,Different input contrasts DF 80 trial1.yaml,complete,1,8:33:13,8:54:57
10,2023-03-02,Different input contrasts LED119 trial1.yaml,complete,1,12:08:15,12:47:13
11,2023-03-02,Different input contrasts DPC raw trial1.yaml,complete,1,5:40:57,6:01:22


In [20]:
import tensorflow as tf

i = tf.compat.v1.train.summary_iterator('/home/hpinkard_waller/models/Different_input_contrasts_DPC_raw_trial1/tensorboard/train/events.out.tfevents.1677808652.waller-fuoco.1383916.0.v2')

for e in i:
    for v in e.summary.value:
        if 'loss' in v.tag:
            print(v.tag, tf.make_ndarray(v.tensor))



epoch_loss 26.819323
epoch_CD123_density_loss 2.490329
epoch_CD14_density_loss 2.7157156
epoch_CD16_density_loss 3.265977
epoch_CD19_density_loss 3.7022707
epoch_CD3_density_loss 2.5418568
epoch_CD45_density_loss 3.397066
epoch_CD56_density_loss 4.8820286
epoch_HLA-DR_density_loss 3.8241508
epoch_loss 14.476684
epoch_CD123_density_loss -0.101375446
epoch_CD14_density_loss -0.5886698
epoch_CD16_density_loss 2.3172555
epoch_CD19_density_loss 2.80519
epoch_CD3_density_loss 2.130203
epoch_CD45_density_loss 2.1086586
epoch_CD56_density_loss 4.1625304
epoch_HLA-DR_density_loss 1.6428846
epoch_loss 10.836905
epoch_CD123_density_loss -0.8626813
epoch_CD14_density_loss -1.9569503
epoch_CD16_density_loss 1.8964145
epoch_CD19_density_loss 2.670666
epoch_CD3_density_loss 2.0419571
epoch_CD45_density_loss 1.9186561
epoch_CD56_density_loss 3.9138992
epoch_HLA-DR_density_loss 1.2149408
epoch_loss 8.923433
epoch_CD123_density_loss -1.2701666
epoch_CD14_density_loss -2.6732383
epoch_CD16_density_loss 1

In [5]:
# read scalars from tensorboard



tensorboard_dir = '/home/hpinkard_waller/models/output_density_type_logistic/tensorboard'


print(read_elasped_time(tensorboard_dir))

events.out.tfevents.1677870387.CIVO-ML-SERVER.12584.0.v2
3:29:13
